In [1]:
import os
import json
from typing import List, Dict, Any, Optional, Union

# Import smolagents components
from smolagents import CodeAgent, HfApiModel, tool




/workspaces/Magentic-AlgoTrading101/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initialize Google Sheet Manager

In [2]:
# Import the GoogleSheetManager class from the conceptual 'google_sheet_manager.py' file
# In a real environment, this would be: `from google_sheet_manager import GoogleSheetManager`
# For this self-contained immersive, we're assuming GoogleSheetManager is defined above
# or is imported from a separate file accessible in the environment.
# For direct execution within this single block, the class would need to be in the same file.

# --- Agent Tool Definitions ---

# IMPORTANT: Ensure GOOGLE_SHEET_CREDENTIALS environment variable is set
# with the JSON content of your Google Service Account key file.
# Replace with the actual ID of your Google Spreadsheet
MY_SPREADSHEET_ID = os.getenv('BUDGET_SPREADSHEET_ID', 'YOUR_SPREADSHEET_ID_HERE') # <--- CHANGE THIS!
# Default sheet name for convenience if not specified in prompt
DEFAULT_SHEET_NAME = 'Sheet1'
# Default row where expenses start (adjust based on your sheet's layout)
DEFAULT_START_EXPENSE_ROW = 7 # Assuming headers and budget info are in rows 1-6

# Initialize GoogleSheetManager once for the agent's tools to use
# This instance will be shared by all the @tool functions.
sheet_manager_instance = None
try:
    # Attempt to initialize the manager.
    # If this script were in a separate file, GoogleSheetManager would be imported.
    # For this immersive, it's conceptually here, assume it's available.
    from google_sheet_manager import GoogleSheetManager # This is how it would look in a separate file
    sheet_manager_instance = GoogleSheetManager()
except ImportError:
    print("Error: 'google_sheet_manager' module not found. Please ensure google_sheet_manager.py is accessible.")
    exit() # Exit if the manager class cannot be imported.
except Exception as e:
    print(f"Error initializing GoogleSheetManager: {e}")
    print("Please ensure GOOGLE_SHEET_CREDENTIALS environment variable is correctly set.")
    exit()

# Exit if manager couldn't be authenticated (e.g., credentials missing/invalid)
if not sheet_manager_instance.service:
    print("Agent tools cannot be initialized without a valid GoogleSheetManager. Exiting.")
    exit()



GoogleSheetManager initialized and authenticated successfully.


### Initializing Tools

In [3]:
@tool
def insert_sheet_row(spreadsheet_id: str = MY_SPREADSHEET_ID, sheet_name: str = DEFAULT_SHEET_NAME, insert_at_row_index: int = DEFAULT_START_EXPENSE_ROW, num_rows: int = 1) -> bool:
    """
    Inserts one or more empty rows into a Google Sheet at a specified 1-based row index.
    Existing rows will be shifted down.
    Args:
        spreadsheet_id (str): The ID of the Google Spreadsheet.
        sheet_name (str): The name of the sheet (default is 'Sheet1').
        insert_at_row_index (int): The 1-based index where new rows will be inserted.
                                   (e.g., 1 to insert at the very top, 7 to insert before row 7).
        num_rows (int): The number of rows to insert (default is 1).
    Returns:
        bool: True if rows were successfully inserted, False otherwise.
    """
    return sheet_manager_instance.insert_empty_row(spreadsheet_id, sheet_name, insert_at_row_index, num_rows)

@tool
def get_current_budget(spreadsheet_id: str = MY_SPREADSHEET_ID, sheet_name: str = DEFAULT_SHEET_NAME) -> Optional[float]:
    """
    Retrieves the total budget value from cell B1 of the specified Google Sheet.
    Args:
        spreadsheet_id (str): The ID of the Google Spreadsheet.
        sheet_name (str): The name of the sheet (default is 'Sheet1').
    Returns:
        Optional[float]: The budget as a float, or None if not found or not numeric.
    """
    return sheet_manager_instance.get_current_budget(spreadsheet_id, sheet_name)

@tool
def get_total_sheet_expenses(spreadsheet_id: str = MY_SPREADSHEET_ID, sheet_name: str = DEFAULT_SHEET_NAME, start_expense_row: int = DEFAULT_START_EXPENSE_ROW) -> Union[int, float, None]:
    """
    Calculates the total sum of all expenses in column C of the specified Google Sheet,
    starting from a given row.
    Args:
        spreadsheet_id (str): The ID of the Google Spreadsheet.
        sheet_name (str): The name of the sheet (default is 'Sheet1').
        start_expense_row (int): The 1-based row number where expense data starts.
    Returns:
        Union[int, float, None]: The total sum of expenses, 0 if no numeric data, or None on error.
    """
    return sheet_manager_instance.get_total_expenses(spreadsheet_id, sheet_name, start_expense_row)

@tool
def get_remaining_sheet_budget(spreadsheet_id: str = MY_SPREADSHEET_ID, sheet_name: str = DEFAULT_SHEET_NAME, start_expense_row: int = DEFAULT_START_EXPENSE_ROW) -> Union[int, float, None]:
    """
    Calculates the remaining budget by subtracting total expenses from the total budget
    in the specified Google Sheet.
    Args:
        spreadsheet_id (str): The ID of the Google Spreadsheet.
        sheet_name (str): The name of the sheet (default is 'Sheet1').
        start_expense_row (int): The 1-based row number where expense data starts.
    Returns:
        Union[int, float, None]: The remaining budget, or None if data cannot be retrieved.
    """
    return sheet_manager_instance.get_remaining_budget(spreadsheet_id, sheet_name, start_expense_row)

@tool
def get_daily_remaining_sheet_budget(spreadsheet_id: str = MY_SPREADSHEET_ID, sheet_name: str = DEFAULT_SHEET_NAME, start_expense_row: int = DEFAULT_START_EXPENSE_ROW) -> Optional[str]:
    """
    Calculates and formats the daily remaining budget.
    If remaining budget is positive, returns "<remaining budget> (number of days left)".
    If remaining budget is negative, returns "<remaining budget>".
    Args:
        spreadsheet_id (str): The ID of the Google Spreadsheet.
        sheet_name (str): The name of the sheet (default is 'Sheet1').
        start_expense_row (int): The 1-based row number where expense data starts.
    Returns:
        Optional[str]: A formatted string representing the daily remaining budget,
                       or None if data cannot be retrieved or dates are invalid.
    """
    return sheet_manager_instance.get_daily_remaining_budget(spreadsheet_id, sheet_name, start_expense_row)

@tool
def get_all_sheet_expenses(spreadsheet_id: str = MY_SPREADSHEET_ID, sheet_name: str = DEFAULT_SHEET_NAME, start_expense_row: int = DEFAULT_START_EXPENSE_ROW, expense_columns: str = 'A:C') -> Optional[List[List[Any]]]:
    """
    Reads and returns all expense data from the specified Google Sheet and columns,
    starting from a given row.
    Args:
        spreadsheet_id (str): The ID of the Google Spreadsheet.
        sheet_name (str): The name of the sheet (default is 'Sheet1').
        start_expense_row (int): The 1-based row number where expense data starts.
                                 (e.g., 7 if headers are up to row 6).
        expense_columns (str): The A1 notation of the columns covering expense data (e.g., 'A:C').
    Returns:
        Optional[List[List[Any]]]: A list of lists representing all expense data, or None on error.
    """
    return sheet_manager_instance.get_all_expenses(spreadsheet_id, sheet_name, start_expense_row, expense_columns)

@tool
def append_rows_to_sheet(spreadsheet_id: str, sheet_name: str, start_row_for_append: int, data_to_append: List[List[Any]]) -> Optional[str]:
    """
    Appends one or more rows of data to a Google Sheet after a specific row number.
    Args:
        spreadsheet_id (str): The ID of the Google Spreadsheet.
        sheet_name (str): The name of the sheet within the spreadsheet (e.g., "Sheet1").
        start_row_for_append (int): The row number *after* which you want to append.
                                    For example, if headers are in row 3, use 4.
        data_to_append (List[List[Any]]): A list of lists representing the rows to append.
                                          Example: [['2025-06-07', 'Groceries', 150]]
    Returns:
        Optional[str]: The updated range string (e.g., 'Sheet1!A10:C11') if successful,
                       or None if an adhesion error occurs.
    """
    return sheet_manager_instance.append_row(spreadsheet_id, sheet_name, start_row_for_append, data_to_append)

@tool
def read_data_from_sheet(spreadsheet_id: str, range_name: str) -> Optional[List[List[Any]]]:
    """
    Reads data from a specified range in a Google Sheet.
    Args:
        spreadsheet_id (str): The ID of the Google Spreadsheet.
        range_name (str): The A1 notation or R1C1 notation of the range to retrieve.
                          (e.g., 'Sheet1!A1:C10' or 'Sheet2!B:B')
    Returns:
        Optional[List[List[Any]]]: A list of lists representing the data in the specified range,
                                  or None if an error occurs or no data is found.
    """
    return sheet_manager_instance.read_sheet_data(spreadsheet_id, range_name)




### Initializing Agent

In [4]:
# --- Example Agent Workflow ---
# IMPORTANT: Ensure GOOGLE_SHEET_CREDENTIALS and BUDGET_SPREADSHEET_ID
# environment variables are set in your Codespace secrets.
# The `MY_SPREADSHEET_ID` and `DEFAULT_START_EXPENSE_ROW` should match your spreadsheet setup.

# Initialize the LLM model for the agent
try:
    agent_model = HfApiModel(model_id="mistralai/Mistral-7B-Instruct-v0.2")
except Exception as e:
    print(f"Failed to initialize agent model: {e}")
    print("Please ensure your Hugging Face API token (HF_TOKEN) is correctly set and you have access to the model.")
    exit()

# List of all tools the agent can use
google_sheets_agent_tools = [
    insert_sheet_row,
    get_current_budget,
    get_total_sheet_expenses,
    get_remaining_sheet_budget,
    get_daily_remaining_sheet_budget,
    get_all_sheet_expenses,
    append_rows_to_sheet, # Existing append tool
    read_data_from_sheet # Existing generic read tool
]

# Create the agent
agent = CodeAgent(
    tools=google_sheets_agent_tools,
    model=agent_model,
    max_steps=20, # Increased max_steps to allow for more complex multi-tool tasks
    name="AdvancedGoogleSheetsAssistant",
    description="An AI assistant capable of managing Google Sheet data. "
                "It can: (1) Insert empty rows, (2) Get the total budget, "
                "(3) Get total expenses, (4) Calculate remaining budget, "
                "(5) Calculate daily remaining budget based on start/end dates (B3/B4), "
                "and (6) Show all recorded expenses. "
                "It primarily operates on the spreadsheet with ID: "
                f"{MY_SPREADSHEET_ID}, assuming 'Sheet1' and expenses starting from row "
                f"{DEFAULT_START_EXPENSE_ROW} in column C, with dates in YYYY-MM-DD format."
)

print(f"\n--- Google Sheets Agent ('{agent.name}') Initialized ---")
print(f"Agent description: {agent.description}")




--- Google Sheets Agent ('AdvancedGoogleSheetsAssistant') Initialized ---
Agent description: An AI assistant capable of managing Google Sheet data. It can: (1) Insert empty rows, (2) Get the total budget, (3) Get total expenses, (4) Calculate remaining budget, (5) Calculate daily remaining budget based on start/end dates (B3/B4), and (6) Show all recorded expenses. It primarily operates on the spreadsheet with ID: 1hGkQHbYKtDfsQvAsgTT6eZe9sPjuxA3MrTVZV1O7RLE, assuming 'Sheet1' and expenses starting from row 7 in column C, with dates in YYYY-MM-DD format.


/workspaces/Magentic-AlgoTrading101/.venv/lib/python3.11/site-packages/smolagents/models.py:1372: FutureWarning: HfApiModel was renamed to InferenceClientModel in version 1.14.0 and will be removed in 1.17.0.
  warnings.warn(


### Insert a  Row

In [5]:
# --- Agent Tasks (Demonstrations) ---

import datetime

# --- Task 1: Add a new expense record ---
print("\n--- Task 1: Add a new expense record ---")

# Define the data for the new expense.
# The format should match your sheet's columns, e.g., Date, Expense Type, Amount.
# Make sure the date format matches what your sheet and tool expect (YYYY-MM-DD).

# Get today's date in YYYY-MM-DD format
today_date = datetime.date.today().strftime("%Y-%m-%d")

expense_type = "Groceries" # Example expense type
amount = 35.75             # Example amount

# The agent needs to know the specific data to append.
# Agent prompt should clearly state the intention and provide the data.
add_expense_prompt = (
    f"Add a new expense record to my Google Sheet (ID: {MY_SPREADSHEET_ID}, Sheet: {DEFAULT_SHEET_NAME}). "
    f"The expense is for '{expense_type}' on '{today_date}' with an amount of £{amount:.2f}. "
    f"Ensure the data is appended correctly to the expense section starting from row {DEFAULT_START_EXPENSE_ROW}."
)

response_add_expense = agent.run(add_expense_prompt)
print(f"Agent's response for Task 1 (Add Expense):\n{response_add_expense}")

# Note: After running this, you might want to immediately run a task
# to get total expenses or remaining budget to confirm the addition.




--- Task 1: Add a new expense record ---


╭──────────────────────────────────── New run - AdvancedGoogleSheetsAssistant ────────────────────────────────────╮
│                                                                                                                 │
│ Add a new expense record to my Google Sheet (ID: 1hGkQHbYKtDfsQvAsgTT6eZe9sPjuxA3MrTVZV1O7RLE, Sheet: Sheet1).  │
│ The expense is for 'Groceries' on '2025-06-23' with an amount of £35.75. Ensure the data is appended correctly  │
│ to the expense section starting from row 7.                                                                     │
│                                                                                                                 │
╰─ HfApiModel - mistralai/Mistral-7B-Instruct-v0.2 ───────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
Provider 'featherless-ai' not supported. Available values: 'auto' or any provider from ['black-forest-labs', 
'cerebras', 'cohere', 'fal-ai', 'fireworks-ai', 'hf-inference', 'hyperbolic', 'nebius', 'novita', 'openai', 
'replicate', 'sambanova', 'together'\].Passing 'auto' (default value) will automatically select the first provider 
available for the model, sorted by the user's order in https://hf.co/settings/inference-providers.

[Step 1: Duration 0.21 seconds]

AgentGenerationError: Error in generating model output:
Provider 'featherless-ai' not supported. Available values: 'auto' or any provider from ['black-forest-labs', 'cerebras', 'cohere', 'fal-ai', 'fireworks-ai', 'hf-inference', 'hyperbolic', 'nebius', 'novita', 'openai', 'replicate', 'sambanova', 'together'].Passing 'auto' (default value) will automatically select the first provider available for the model, sorted by the user's order in https://hf.co/settings/inference-providers.

### Get Current Budget

In [ ]:
# Task 2: Get current budget
print("\n--- Task 2: Find Current Budget ---")
current_budget_prompt = f"What is the total budget from my Google Sheet (ID: {MY_SPREADSHEET_ID}, Sheet1)?"
response_budget = agent.run(current_budget_prompt)
print(f"Agent's response for Task 2:\n{response_budget}")



### Get Total Expenses and Remaining Budget

In [ ]:
# Task 3: Get total expenses and remaining budget
print("\n--- Task 3: Find Total and Remaining Budget ---")
total_remaining_budget_prompt = (
    f"Calculate my total expenses and then tell me my remaining budget in my Google Sheet "
    f"(ID: {MY_SPREADSHEET_ID}, Sheet1, expenses starting row {DEFAULT_START_EXPENSE_ROW})."
)
response_remaining_budget = agent.run(total_remaining_budget_prompt)
print(f"Agent's response for Task 3:\n{response_remaining_budget}")



### Calculate Daily Budget

In [ ]:
# Task 4: Calculate daily budget remaining
print("\n--- Task 4: Calculate Daily Remaining Budget ---")
daily_budget_prompt = (
    f"Calculate the daily remaining budget for my current period. "
    f"The spreadsheet ID is {MY_SPREADSHEET_ID}, Sheet1, expenses start row {DEFAULT_START_EXPENSE_ROW}. "
    f"The start date is in B3 and end date is in B4. Dates are YYYY-MM-DD."
)
response_daily_budget = agent.run(daily_budget_prompt)
print(f"Agent's response for Task 4:\n{response_daily_budget}")



### Show All Expenses

In [ ]:
# Task 5: Show all expenses
print("\n--- Task 5: Show All Expenses ---")
all_expenses_prompt = (
    f"Show me all my expenses from Sheet1 of my Google Sheet (ID: {MY_SPREADSHEET_ID}), "
    f"assuming they start from row {DEFAULT_START_EXPENSE_ROW} and are in columns A to C."
)
response_all_expenses = agent.run(all_expenses_prompt)
print(f"Agent's response for Task 5:\n{response_all_expenses}")

print("\n--- Agent Workflow Complete ---")